# Fine tuning

In [19]:
config = {
    "lr" : 1e-6  
    }

## Set up

In [20]:
!nvidia-smi # to see what GPU you have

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Mon May  1 18:25:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   38C    P0    26W /  70W |   1247MiB / 15360MiB |      0%    

In [21]:
# !pip install wandb --quiet
# !pip install torchsummaryX -q
# !pip install mutagen
# !pip install jiwer
# !pip install git+https://github.com/openai/whisper.git 
# # on Ubuntu or Debian
# !sudo apt update && sudo apt install ffmpeg
# !pip install -qqq evaluate==0.2.2
# !pip install librosa

In [22]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [23]:
# print(os.environ['CUDA_PATH'])

In [24]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from tqdm import tqdm

import torchaudio.transforms as tat

# from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

from mutagen.mp3 import MP3
import jiwer

import wandb
import evaluate
import librosa
import warnings
warnings.filterwarnings('ignore')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

if DEVICE == 'cuda':
    dtype = torch.float16
else:
    dtype = torch.float32

Device:  cuda


## Load the pretrained Whisper model and tokenizer

In [25]:
import whisper

model = whisper.load_model("tiny")
tokenizer = whisper.tokenizer.get_tokenizer(model.is_multilingual, language="en", task="transcribe")

In [26]:
# hard-coded audio hyperparameters
# SAMPLE_RATE = 16000
# N_FFT = 400
# N_MELS = 80
# HOP_LENGTH = 160
# CHUNK_LENGTH = 30
# N_SAMPLES = CHUNK_LENGTH * SAMPLE_RATE  # 480000 samples in a 30-second chunk
# N_FRAMES = whisper.utils.exact_div(N_SAMPLES, HOP_LENGTH)  # 3000 frames in a mel spectrogram input

# N_SAMPLES_PER_TOKEN = HOP_LENGTH * 2  # the initial convolutions has stride 2
# FRAMES_PER_SECOND = whisper.utils.exact_div(SAMPLE_RATE, HOP_LENGTH)  # 10ms per audio frame
# TOKENS_PER_SECOND = whisper.utils.exact_div(SAMPLE_RATE, N_SAMPLES_PER_TOKEN)  # 20ms per audio token

In [27]:
torch.cuda.empty_cache()
gc.collect()

17

In [28]:
# y, sr = librosa.load('/home/ubuntu/cv-corpus-13.0-2023-03-09/en/clips/common_voice_en_663624.mp3', sr=48000)
# y_16k = librosa.resample(y, orig_sr=sr, target_sr=16000)

## Dataloader

In [29]:
BATCH_SIZE = 10 # Increase if your device can handle it
num_updates = 200
val_updates = 50

In [30]:
# f = open("log.txt","w")
# f.close()

In [31]:
# tokenizer.encode("!")

In [32]:
# the size of tensor a (2703) must match the size of tensor b (448) at non-singleton dimension 1

In [33]:
# # mfcc = whisper.audio.log_mel_spectrogram("/home/ubuntu/cv-corpus-13.0-2023-03-09/en/clips/common_voice_en_17888668.mp3",padding=N_SAMPLES)
# # mfcc = whisper.audio.pad_or_trim(mfcc, N_FRAMES).to(DEVICE).to(dtype)
# # print(mfcc.shape)
# filepath = '/home/ubuntu/cv-corpus-13.0-2023-03-09/en/clips/common_voice_en_20266264.mp3'
# orig_audio, sr = librosa.load(filepath, sr=48000)
# whisper_audio = librosa.resample(orig_audio, orig_sr=sr, target_sr=16000)
# # mfcc = whisper.audio.log_mel_spectrogram(whisper_audio)
# # mfcc = whisper.audio.pad_or_trim(mfcc)
# mfcc = whisper.audio.log_mel_spectrogram(whisper_audio,padding = 480000)
# mfcc = whisper.audio.pad_or_trim(mfcc, 3000).to(DEVICE).to(dtype)

In [34]:
# mfcc = mfcc.reshape(1,80,3000)
# h, lh = asrmodel(mfcc, torch.zeros(1,10,dtype=torch.long).to(DEVICE))

In [35]:
class AudioDataloader(torch.utils.data.DataLoader):

    # For this homework, we give you full flexibility to design your data set class.
    
    def __init__(self, partition, root, tokenizer, batch_size, shuffle): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''
        self.df = pd.read_csv('common_voice_'+partition+'.csv')
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.df_len = len(self.df)
        self.root = root
        self.shuffle = shuffle
        

    def __len__(self):
        
        return round((self.df_len//self.batch_size)*0.95)
    
    def __iter__(self):
        if self.shuffle:
            self.df = self.df.sample(frac = 1)
        num_batches = self.__len__()
        batch_idx = 0
        batch_mfccs = []
        batch_transcripts = []
        batch_probs = []
        lengths_mfcc = []
        lengths_transcript = []
        for index, entry in self.df.iterrows():
            filepath = self.root + entry['path']
            audio = MP3(filepath)
            if audio.info.length < 28:
                # f.write(filepath+"\n")
                transcript = torch.tensor(tokenizer.encode(entry['sentence']))
                prob = entry['probs']
                orig_audio, sr = librosa.load(filepath, sr=48000)
                whisper_audio = librosa.resample(orig_audio, orig_sr=sr, target_sr=16000)
                # mfcc = whisper.audio.log_mel_spectrogram(whisper_audio)
                # mfcc = whisper.audio.pad_or_trim(mfcc)
                mfcc = whisper.audio.log_mel_spectrogram(whisper_audio,padding = 480000)
                mfcc = whisper.audio.pad_or_trim(mfcc, 3000).to(DEVICE).to(dtype)
                batch_mfccs.append(mfcc)
                lengths_mfcc.append(len(mfcc))
                batch_transcripts.append(transcript)
                lengths_transcript.append(len(transcript))
                batch_probs.append(prob)
            if len(batch_probs)== self.batch_size:
                # f.write("new batch\n")
                batch_idx += 1
                batch_mfcc_pad = pad_sequence(batch_mfccs, batch_first= True) # TODO
                # max_label_len = max(lengths_transcript)
                # batch_transcript_pad = [np.pad(lab, (0, max_label_len - lab_len), 'constant', constant_values=-100) for lab, lab_len in zip(batch_transcripts, lengths_transcript)]
                batch_transcript_pad = pad_sequence(batch_transcripts, batch_first= True) # TODO
                batch_transcript_pad_ent = pad_sequence(batch_transcripts, batch_first= True, padding_value = 0) # TODO
                yield batch_mfcc_pad, batch_transcript_pad, torch.tensor(batch_probs), torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript),batch_transcript_pad_ent
                batch_mfccs = []
                batch_transcripts = []
                batch_probs = []
                lengths_mfcc = []
                lengths_transcript = []
        self.length = len(self.mfccs)

In [36]:
# Create objects for the dataset class
root = '/home/ubuntu/cv-corpus-13.0-2023-03-09/en/clips/'
# train_data = AudioDataset('train',root, tokenizer) #TODO
# val_data = AudioDataset('validation',root, tokenizer) # TODO : You can either use the same class with some modifications or make a new one :)

# # Do NOT forget to pass in the collate function as parameter while creating the dataloader
# train_loader = torch.utils.data.DataLoader(
#     dataset     = train_data, 
#     num_workers = 0,
#     batch_size  = BATCH_SIZE, 
# #     pin_memory  = True,
#     shuffle     = True,
#     collate_fn  = train_data.collate_fn
# )
# val_loader = torch.utils.data.DataLoader(
#     dataset     = val_data,
#     num_workers = 0,
#     batch_size  = BATCH_SIZE, 
#     # pin_memory  = True,
#     shuffle     = False,
#     collate_fn  = val_data.collate_fn
# )

# print("Batch size: ", BATCH_SIZE)
# print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
# print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
train_loader = AudioDataloader('train', root, tokenizer, BATCH_SIZE, shuffle = True)
val_loader = AudioDataloader('validation', root, tokenizer, BATCH_SIZE, shuffle = True)
print("train batches", len(train_loader))
print("val batches", len(val_loader))

train batches 59530
val batches 14882


In [50]:
# sanity check
x, y, p, lx, ly, yc = next(iter(train_loader))
print(x.shape, y.shape, p.shape, lx.shape, ly.shape)
# print(x,y,p,lx,ly)
x, y, p, lx, ly, yc = next(iter(train_loader))
print(x.shape, y.shape, p.shape, lx.shape, ly.shape)
# print(x,y,p,lx,ly)

torch.Size([10, 80, 3000]) torch.Size([10, 23]) torch.Size([10]) torch.Size([10]) torch.Size([10])
torch.Size([10, 80, 3000]) torch.Size([10, 24]) torch.Size([10]) torch.Size([10]) torch.Size([10])


## Training Model

In [37]:
class ASRModel(torch.nn.Module):

    def __init__(self, model):
        super().__init__()
        self.options = whisper.decoding.DecodingOptions(fp16 = True,temperature=0.0,without_timestamps=True)
        self.asr = model
        
    
    def forward(self, x, y):
        # print("x type",type(x))
        # print("x shape",x.shape)
        # print("opt type",type(self.options))
        # print("opt shape",self.options.shape)
        # print("length",lengths_x)
        audio_features = self.asr.encoder(x)
        results = self.asr.decoder(y, audio_features)
        # print(results.shape,results)
        results = results.log_softmax(2)
        # print(results.shape,results)
        # out = [torch.LongTensor(result.tokens).to(DEVICE) for result in results]
        # out = torch.nn.utils.rnn.pad_sequence(out, batch_first=True, padding_value=0.0)
        lx = self.asr.decode(x,self.options)
        # print("orig len", len(out))
        lx = list(map(lambda x: len(x.tokens), lx))
        # seq_unpacked, lens_unpacked = pad_packed_sequence(out, batch_first = True)
        # print("new len",len(out))
        lx = torch.LongTensor(lx).to(DEVICE)
        # return out, lengths_x
        # return seq_unpacked, lens_unpacked
        # print(type(out))
        # print(out.shape)
        # out = out.double()
        # return out, lengths_x
        return results,lx

## Criterion, optimizer, scheduler

In [38]:
# criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True) # Define CTC loss as the criterion. How would the losses be reduced?
criterion = torch.nn.CrossEntropyLoss(reduction = 'none', ignore_index = 0)
optimizer =  torch.optim.SGD(model.parameters(),lr = config["lr"]) # What goes in here?

scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1, end_factor=0.5, total_iters=len(train_loader), last_epoch=- 1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2) #TODO
# metric = evaluate.load("wil")
# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

## Train and validate functions

In [39]:
def validate_model(model, val_loader, val_batches):

    model.eval()
    batch_bar = tqdm(total=val_batches, dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    val_wil = 0

    for i, data in enumerate(val_loader):
        if i == val_batches:
            break
        x, y, p, lx, ly, yc = data
        x, y, p, yc = x.to(DEVICE), y.to(DEVICE), p.to(DEVICE), yc.to(DEVICE)

        with torch.inference_mode():
            h,lh = model(x, y)
            h = torch.permute(h, (0, 2, 1))
            loss = criterion(h, yc)
            loss = torch.sum(loss,dim=1)
            loss = torch.mean(loss)
            # loss = torch.mean(loss)
        total_loss += float(loss)
        # val_wil += wil(h,y)
        
        ht = model.asr.decode(x,model.options)
        ht = list(map(lambda x: x.tokens, ht))
        text_x,text_y = [],[]
        for j in range(len(ht)):
            val_wil += jiwer.wil(tokenizer.decode(ht[j]), tokenizer.decode(y[j]))/len(ht)            

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), wil="{:.04f}".format(float(val_wil / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, yc, ht, text_x, text_y, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/ val_batches
    avg_wil = val_wil/ val_batches
    return total_loss, avg_wil

## Training setup

In [40]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1], 
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

## wandb

In [41]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
best_wil = 1 # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/home/ubuntu/11785-ASR-Project/finetune/reg_epoch_model_05_01.checkpoint"#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "/home/ubuntu/11785-ASR-Project/finetune/reg_best_model_05_01.checkpoint"#TODO set best model path 

In [42]:
# checkpoint = torch.load(best_model_path)
# print(checkpoint['val_wil'])

In [43]:
wandb.login(key="a5b7420abbe354e6d0b2f5554b97ee11f327fc92") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Currently logged in as: shilinm (deeper_learners). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

In [44]:
# load_model(best_model_path, asrmodel, metric= 'val_wil', optimizer= None, scheduler= None)

In [45]:
# Create your wandb run
run = wandb.init(
    name = "reg-test-run-clip", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # id = 'oshmtgys', #Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "whisper-finetune", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

In [60]:
# run.finish()

## Train

In [46]:
asrmodel = ASRModel(model)

In [47]:
# val_loss, val_wil = validate_model(asrmodel,val_loader, 10)
# print("start val loss", val_loss, "start val wil", val_wil)

In [48]:
torch.cuda.empty_cache()
gc.collect()

567

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop
# best_wil = 0.4252399123828323
for epoch in range(1):

    total_loss = 0
    batch_bar = tqdm(total=num_updates, dynamic_ncols=True, leave=False, position=0, desc='Train')
    asrmodel.train()
    for i, data in enumerate(train_loader):
        try:
            if (i+1)%num_updates==0:
                batch_bar.close()
                train_loss = total_loss / num_updates
                val_loss, val_wil = validate_model(asrmodel,val_loader, val_updates)
                curr_lr = float(optimizer.param_groups[0]['lr'])
                print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
                print("\tVal Wil {:.04f}\t Val Loss {:.04f}".format(val_wil, val_loss))
                wandb.log({
                    'train_loss': train_loss,  
                    'val_wil': val_wil, 
                    'val_loss': val_loss, 
                    'lr'        : curr_lr,
                    'epoch'     : i//num_updates,
                })
                save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], i//num_updates, epoch_model_path)
                if val_wil <= best_wil:
                    best_wil = val_wil
                    save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], i//num_updates, best_model_path)
                    print("saving best model")
                total_loss = 0
                batch_bar = tqdm(total=num_updates, dynamic_ncols=True, leave=False, position=0, desc='Train')

            optimizer.zero_grad()

            x, y, p, lx, ly, yc = data
            x, y, p, yc = x.to(DEVICE), y.to(DEVICE),p.to(DEVICE), yc.to(DEVICE)

            with torch.cuda.amp.autocast():     
                h, lh = asrmodel(x, y)
                h = torch.permute(h, (0, 2, 1))
                loss = criterion(h, yc)
                loss = torch.sum(loss,dim=1)
                # loss /= p
                loss = torch.mean(loss)

            total_loss += loss.item()

            batch_bar.set_postfix(
                loss="{:.04f}".format(float(total_loss / (i + 1))),
                lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

            batch_bar.update() # Update tqdm bar

            # Another couple things you need for FP16. 
            scaler.scale(loss).backward() # This is a replacement for loss.backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=1)
            scaler.step(optimizer) # This is a replacement for optimizer.step()
            scaler.update() # This is something added just for FP16
            scheduler.step()

            del x, y, lx, ly, h, lh, yc, loss 
            torch.cuda.empty_cache()
        except:
            print("exception occured")

    batch_bar.close() # You need this to close the tqdm bar
    val_loss, val_wil = validate_model(asrmodel,val_loader, val_updates)
    save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], epoch, epoch_model_path)
    if val_wil <= best_wil:
        best_wil = val_wil
        save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], epoch, best_model_path)
        print("saving best model")
    
# run.finish()

	Train Loss 192.6933	 Learning Rate 0.0000010
	Val Wil 0.4553	 Val Loss 187.7917
saving best model


	Train Loss 191.1729	 Learning Rate 0.0000010
	Val Wil 0.4445	 Val Loss 185.2332
saving best model


	Train Loss 189.6922	 Learning Rate 0.0000010
	Val Wil 0.4767	 Val Loss 187.7363


	Train Loss 188.3834	 Learning Rate 0.0000010
	Val Wil 0.4583	 Val Loss 189.8207


	Train Loss 182.3511	 Learning Rate 0.0000010
	Val Wil 0.4623	 Val Loss 188.2877


...

	Train Loss 122.0770	 Learning Rate 0.0000009
	Val Wil 0.4321	 Val Loss 119.4825


	Train Loss 118.6865	 Learning Rate 0.0000009
	Val Wil 0.4380	 Val Loss 117.6606


	Train Loss 116.6916	 Learning Rate 0.0000009
	Val Wil 0.4592	 Val Loss 115.3483


	Train Loss 114.5651	 Learning Rate 0.0000009
	Val Wil 0.4710	 Val Loss 113.0477


Train:  57%|████▌   | 114/200 [01:40<01:11,  1.20it/s, loss=1.4661, lr=0.000001]

# Playing with the model

In [25]:
# tokenizer = whisper.tokenizer.get_tokenizer(model.is_multilingual, language="en", task="transcribe")
# t = tokenizer.encode("Please call Stella. Ask her to bring these things with her from the store: Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids. She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.")
# print(len(t),t)

# # import torchaudio
# # def load_wave(wave_path, sample_rate:int=16000) -> torch.Tensor:
# #     waveform, sr = torchaudio.load(wave_path, normalize=True)
# #     if sample_rate != sr:
# #         waveform = tat.Resample(sr, sample_rate)(waveform)
# #     return waveform
# # audio = load_wave('/content/data/recordings/recordings/english1.mp3', sample_rate=16000)
# # audio = whisper.pad_or_trim(audio.flatten())
# # mel = whisper.log_mel_spectrogram(audio)
# mel = whisper.audio.log_mel_spectrogram('/data/home/ubuntu/cv-corpus-13.0-2023-03-09/en/clips/common_voice_en_22930344.mp3',padding=N_SAMPLES)
# mel = whisper.audio.pad_or_trim(mel, N_FRAMES).to('cuda').to(torch.float32)
# options = whisper.decoding.DecodingOptions(fp16 = False,temperature=0.0)
# m = model.decode(mel,options)
# print(len(m.tokens),m.tokens)
# # d = predict(model, mel)
# # print(d)

79 [16216, 818, 45073, 13, 12320, 720, 281, 1565, 613, 721, 365, 720, 490, 264, 3531, 25, 11678, 36316, 295, 4451, 5756, 24494, 11, 1732, 5060, 1061, 17243, 295, 3344, 5399, 11, 293, 1310, 257, 13288, 337, 720, 3708, 6085, 13, 220, 492, 611, 643, 257, 1359, 5900, 12650, 293, 257, 955, 12058, 17259, 337, 264, 2301, 13, 1240, 393, 19555, 613, 721, 666, 1045, 2182, 10405, 11, 293, 321, 486, 352, 1677, 720, 10579, 412, 264, 3847, 5214, 13]
11 [50364, 18951, 2405, 307, 1071, 1021, 39230, 294, 43120, 13, 50564]


In [11]:
tokenizer.decode([50364, 2555, 818, 45073, 13, 12320, 720, 281, 1565, 613, 721, 365, 720, 490, 264, 3531, 13, 11678, 50608, 50608, 36316, 295, 4451, 5756, 24494, 11, 1732, 5060, 1061, 17243, 295, 3344, 5399, 11, 293, 1310, 257, 13288, 337, 720, 50880, 50880, 3708, 6085, 13, 492, 611, 643, 257, 1359, 5900, 12650, 293, 257, 955, 12058, 17259, 337, 264, 2301, 13, 51120, 51120, 1240, 393, 19555, 613, 721, 666, 1045, 2182, 10405, 293, 321, 486, 352, 1677, 720, 10579, 51390, 51390, 412, 264, 3847, 5214, 13, 51440])

' Please call Stella. Ask her to bring these things with her from the store. Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob. We also need a small plastic snake and a big toy frog for the kids. She can scoop these things into three red bags and we will go meet her Wednesday at the train station.'

In [12]:
tokenizer.decode([16216, 818, 45073, 13, 220, 12320, 720, 281, 1565, 613, 721, 365, 720, 490, 264, 3531, 25, 220, 11678, 36316, 295, 4451, 5756, 24494, 11, 1732, 5060, 1061, 17243, 295, 3344, 5399, 11, 293, 1310, 257, 13288, 337, 720, 3708, 6085, 13, 220, 492, 611, 643, 257, 1359, 5900, 12650, 293, 257, 955, 12058, 17259, 337, 264, 2301, 13, 220, 1240, 393, 19555, 613, 721, 666, 1045, 2182, 10405, 11, 293, 321, 486, 352, 1677, 720, 10579, 412, 264, 3847, 5214, 13])

'Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.'

In [ ]:
tokenizer = whisper.tokenizer.get_tokenizer(model.is_multilingual, language="en", task="transcribe")

In [ ]:
def predict(model, tokenizer, mel):
  decode_options = dict()
  decode_options["fp16"] = (DEVICE == 'cuda')
  if decode_options["fp16"]:
    dtype = torch.float16
  else:
    dtype = torch.float32
  logprob_threshold = -1.0
  no_speech_threshold = 0.6
  decode_options["language"] = "en"


  content_frames = mel.shape[-1] - N_FRAMES

  def decode_with_fallback(segment: torch.Tensor) -> whisper.decoding.DecodingResult:
      kwargs = {**decode_options}
      options = whisper.decoding.DecodingOptions(**kwargs, temperature=0.0)
      decode_result = model.decode(segment, options)
      return decode_result

  seek = 0
  input_stride = whisper.utils.exact_div(
      N_FRAMES, model.dims.n_audio_ctx
  )  # mel frames per output token: 2
  time_precision = (
      input_stride * HOP_LENGTH / SAMPLE_RATE
  )  # time per output token: 0.02 (seconds)
  all_tokens = []
  all_segments = []
  prompt_reset_since = 0

  initial_prompt_tokens = []

  def new_segment(
      *, start: float, end: float, tokens: torch.Tensor, result: whisper.decoding.DecodingResult
  ):
      tokens = tokens.tolist()
      text_tokens = [token for token in tokens if token < tokenizer.eot]
      return {
          "seek": seek,
          "start": start,
          "end": end,
          "text": tokenizer.decode(text_tokens),
          "tokens": tokens,
          "temperature": result.temperature,
          "avg_logprob": result.avg_logprob,
          "compression_ratio": result.compression_ratio,
          "no_speech_prob": result.no_speech_prob,
      }

# show the progress bar when verbose is False (if True, transcribed text will be printed)

  while seek < content_frames:
      time_offset = float(seek * HOP_LENGTH / SAMPLE_RATE)
      mel_segment = mel[:, seek : seek + N_FRAMES]
      segment_size = min(N_FRAMES, content_frames - seek)
      segment_duration = segment_size * HOP_LENGTH / SAMPLE_RATE
      mel_segment = whisper.audio.pad_or_trim(mel_segment, N_FRAMES).to(model.device).to(dtype)

      decode_options["prompt"] = all_tokens[prompt_reset_since:]
      result: whisper.decoding.DecodingResult = decode_with_fallback(mel_segment)
      tokens = torch.tensor(result.tokens)

      if no_speech_threshold is not None:
          # no voice activity check
          should_skip = result.no_speech_prob > no_speech_threshold
          if (
              logprob_threshold is not None
              and result.avg_logprob > logprob_threshold
          ):
              # don't skip if the logprob is high enough, despite the no_speech_prob
              should_skip = False

          if should_skip:
              seek += segment_size  # fast-forward to the next segment boundary
              continue

      previous_seek = seek
      current_segments = []

      timestamp_tokens: torch.Tensor = tokens.ge(tokenizer.timestamp_begin)
      single_timestamp_ending = timestamp_tokens[-2:].tolist() == [False, True]

      consecutive = torch.where(timestamp_tokens[:-1] & timestamp_tokens[1:])[0]
      consecutive.add_(1)
      if len(consecutive) > 0:
          # if the output contains two consecutive timestamp tokens
          slices = consecutive.tolist()
          if single_timestamp_ending:
              slices.append(len(tokens))

          last_slice = 0
          for current_slice in slices:
              sliced_tokens = tokens[last_slice:current_slice]
              start_timestamp_pos = (
                  sliced_tokens[0].item() - tokenizer.timestamp_begin
              )
              end_timestamp_pos = (
                  sliced_tokens[-1].item() - tokenizer.timestamp_begin
              )
              current_segments.append(
                  new_segment(
                      start=time_offset + start_timestamp_pos * time_precision,
                      end=time_offset + end_timestamp_pos * time_precision,
                      tokens=sliced_tokens,
                      result=result,
                  )
              )
              last_slice = current_slice

          if single_timestamp_ending:
              # single timestamp at the end means no speech after the last timestamp.
              seek += segment_size
          else:
              # otherwise, ignore the unfinished segment and seek to the last timestamp
              last_timestamp_pos = (
                  tokens[last_slice - 1].item() - tokenizer.timestamp_begin
              )
              seek += last_timestamp_pos * input_stride
      else:
          duration = segment_duration
          timestamps = tokens[timestamp_tokens.nonzero().flatten()]
          if (
              len(timestamps) > 0
              and timestamps[-1].item() != tokenizer.timestamp_begin
          ):
              # no consecutive timestamps but it has a timestamp; use the last one.
              last_timestamp_pos = (
                  timestamps[-1].item() - tokenizer.timestamp_begin
              )
              duration = last_timestamp_pos * time_precision

          current_segments.append(
              new_segment(
                  start=time_offset,
                  end=time_offset + duration,
                  tokens=tokens,
                  result=result,
              )
          )
          seek += segment_size

      if result.temperature > 0.5:
          # do not feed the prompt tokens if a high temperature was used
          prompt_reset_since = len(all_tokens)

      # if a segment is instantaneous or does not contain text, clear it
      for i, segment in enumerate(current_segments):
          if segment["start"] == segment["end"] or segment["text"].strip() == "":
              segment["text"] = ""
              segment["tokens"] = []
              segment["words"] = []

      all_segments.extend(
          [
              {"id": i, **segment}
              for i, segment in enumerate(
                  current_segments, start=len(all_segments)
              )
          ]
      )
      all_tokens.extend(
          [token for segment in current_segments for token in segment["tokens"]]
      )


  return tokenizer.decode(all_tokens)

In [37]:
text_transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
    jiwer.Strip(),
    jiwer.ReduceToListOfListOfWords()
]) 

def wer(target, output):
  return jiwer.wer(
    target, 
    output, 
    truth_transform=text_transformation, 
    hypothesis_transform=text_transformation)
  
def wil(target, output):
  return jiwer.wil(
    target, 
    output, 
    truth_transform=text_transformation, 
    hypothesis_transform=text_transformation)


In [38]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, cfx, p, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():     
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

NameError: name 'LABELS' is not defined

# Baseline

## Fetching the Dataset

In [31]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /home/ubuntu/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"UserName","key":"Key"}') 
#     # Put your kaggle username & key here
with open("/home/ubuntu/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"sma2023","key":"6c819f763f537a6b8bbb60cb11520dbf"}') 
    # Put your kaggle username & key here

!chmod 600 /home/ubuntu/.kaggle/kaggle.json

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
mkdir: cannot create directory ‘/home/ubuntu/.kaggle’: File exists
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin

In [33]:
!mkdir /home/ubuntu/GMUdata

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'


In [34]:
!kaggle datasets download -d rtatman/speech-accent-archive
!unzip -qo 'speech-accent-archive.zip' -d '/home/ubuntu/GMUdata'

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
speech-accent-archive.zip: Skipping, found more recently modified local copy (use --force to force download)
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'


## Define the two metrics

In [ ]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.8 MB/s eta 0:00:00


In [ ]:
import jiwer

In [ ]:
jiwer.wer("the cat", "cat the")

1.0

In [ ]:
f = open("/home/ubuntu/GMUdata/reading-passage.txt")
target = ""
for line in f:
  target += line

text_transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
    # jiwer.ReduceToSingleSentence(),
    jiwer.Strip(),
    jiwer.SubstituteRegexes({r"6": r"six", r"5": r"five", r"3": r"three"}),
    jiwer.ReduceToListOfListOfWords()
]) 

def wer(output):
  return jiwer.wer(
    target, 
    output, 
    truth_transform=text_transformation, 
    hypothesis_transform=text_transformation)
  
def wil(output):
  return jiwer.wil(
    target, 
    output, 
    truth_transform=text_transformation, 
    hypothesis_transform=text_transformation)


## Transcribe and recording the data

In [17]:
# open the speaker_all.csv
import pandas as pd
from tqdm.auto import tqdm

In [36]:
load_model(best_model_path, asrmodel, metric= 'val_wil', optimizer= None, scheduler= None)

[ASRModel(
   (asr): Whisper(
     (encoder): AudioEncoder(
       (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
       (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
       (blocks): ModuleList(
         (0): ResidualAttentionBlock(
           (attn): MultiHeadAttention(
             (query): Linear(in_features=384, out_features=384, bias=True)
             (key): Linear(in_features=384, out_features=384, bias=False)
             (value): Linear(in_features=384, out_features=384, bias=True)
             (out): Linear(in_features=384, out_features=384, bias=True)
           )
           (attn_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
           (mlp): Sequential(
             (0): Linear(in_features=384, out_features=1536, bias=True)
             (1): GELU(approximate='none')
             (2): Linear(in_features=1536, out_features=384, bias=True)
           )
           (mlp_ln): LayerNorm((384,), eps=1e-05, elementwise_a

In [18]:
speakers = pd.read_csv('../whisper_baseline/results.csv')
speakers['new_wer'] = 1.0
speakers['new_wil'] = 1.0
cnt = 0
batch_bar   = tqdm(total=len(speakers), dynamic_ncols=True, leave=False, position=0)
for index, row in speakers.iterrows():
  if row['file_missing?']==False:
    file_name = row['filename']
    transcription = whisper.transcribe(model = model, audio = '/home/ubuntu/GMUdata/recordings/recordings/'+file_name+'.mp3', fp16=False, language = 'en')['text']
    speakers.at[index,'new_wer'] = wer(transcription)
    speakers.at[index,'new_wil'] = wil(transcription)
    # print("wer", speakers.at[index,'wer'], "new_new", speakers.at[index,'new_wer'] , "wil", speakers.at[index,'wil'], "new_wil", speakers.at[index,'new_wil']) 
  batch_bar.update()
batch_bar.close()
speakers.to_csv('is_results_06_01_best.csv')


  0%|                                                  | 0/2172 [00:00<?, ?it/s]

In [31]:
transcription = whisper.transcribe(model = model, audio = '/home/ubuntu/GMUdata/recordings/recordings/yiddish5.mp3', fp16=False)['text']
print(transcription)

 Please call Stella, ask her to bring these things with her from the store. Six points of fresh snow peas, five sticks of blue cheese, maybe a snack for a broader barbe. We also need a small plastic snake and a big toy frag for the kids. She can scoop these things into three red bags and we will go meet her Wednesday at a train station.


In [32]:
transcription = whisper.transcribe(model = model, audio = '/home/ubuntu/GMUdata/recordings/recordings/bambara5.mp3', fp16=False)['text']
print(transcription)

 Please call Stella, ask her to bring these things with her from Vestor. 6 spoons of fresh snow peas, 5 teak slabs of blue sheets, and maybe a snake for her brother Bob. We also need a small plastic snake and big to eat frog from the kids. For the kids. She can scoop these things into three red bags and we will go meet her when they areGHT to enjoy the whole trip.
